# Usando Modelos prontos

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Instalando a Hugging Face PyTorch Interface para o Bert
!pip install -q transformers
!pip install -q torch
!pip install -q sentencepiece
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.2 MB/s eta 0:00:00


In [3]:
import transformers

from transformers import pipeline

#Importa os módulos pré-treinados (pretrained BERT tokenizer e BERT model)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from transformers import BartTokenizer, BartForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import sentencepiece

import torch

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!nvidia-smi

Thu Nov 23 12:31:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Análise de Sentimentos

Multilingual XLM-Roberta model sequence classifier fine tunned and based on Cardiff NLP Group sentiment classification model.

In [5]:
#Usando um pipeline como um high-level helper
model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
model = pipeline("sentiment-analysis", model = model_name, tokenizer = model_name)
model("Eu estou gostando muito desse curso.")


config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

[{'label': 'Positive', 'score': 0.9928063154220581}]

In [6]:
# Carregando o modelo diretamente
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name)

#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)

input_text = "Eu estou gostando muito desse curso."

# Tokenização manual
inputs = tokenizer(input_text, return_tensors="pt")
# Inferência manual
outputs = model(**inputs)

# Obter a classe predita (índice da classe com maior pontuação)
# outputs.logits: tensor com as pontuações não normalizadas ou scores
# torch.argmax: retorna o índice do valor máximo
# aplicada ao tensor logits obtem o índice da classe que tem o maior score (a classe com maior probabilidade).
predicted_class_idx = torch.argmax(outputs.logits).item()

class_labels = ["Negativo", "Neutro", "Positivo"]
predicted_class = class_labels[predicted_class_idx]
score =  torch.max(outputs.logits).item()
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
prob = torch.max(predictions).item()

result = {
          "label": predicted_class,
          "tensor": outputs.logits,
          "logits": score,
          "predictions": predictions,
          "prob": prob
          }


for key, value in result.items():
    print(f"{key}: {value}")

label: Positivo
tensor: tensor([[-2.8015, -1.1371,  3.9636]], grad_fn=<AddmmBackward0>)
logits: 3.9636178016662598
predictions: tensor([[0.0011, 0.0060, 0.9928]], grad_fn=<SoftmaxBackward0>)
prob: 0.9928063154220581


## Classificação Zero-Shot

BART model pre-trained on English language. bart-large-mnli is the checkpoint for bart-large after being trained on the MultiNLI (MNLI) dataset.

In [7]:
model_name = "facebook/bart-large-mnli"
model = pipeline("zero-shot-classification", model = model_name)
model("Este é um curso sobre Transformers para Processamento de Linguagem Natural.",
      candidate_labels = ["Educação", "Política", "Business"])

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'sequence': 'Este é um curso sobre Transformers para Processamento de Linguagem Natural.',
 'labels': ['Educação', 'Política', 'Business'],
 'scores': [0.49541136622428894, 0.41976335644721985, 0.08482524007558823]}

In [8]:
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name)

#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)

input_text = "Este é um curso sobre Transformers para Processamento de Linguagem Natural."
candidate_labels = ["Educação", "Política", "Business"]

# Tokenização manual
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model(**inputs)

# Obter a pontuação para cada classe candidata
class_scores = outputs.logits.squeeze().tolist()
# Criar um dicionário associando rótulos candidatos às pontuações
label_scores = dict(zip(candidate_labels, class_scores))

result = {
          "sequence": input_text,
          "tensor": outputs.logits,
          "prob": class_scores,
          "labels": candidate_labels,
          "scores": label_scores}

for key, value in result.items():
    print(f"{key}: {value}")

sequence: Este é um curso sobre Transformers para Processamento de Linguagem Natural.
tensor: tensor([[ 0.0407,  0.7992, -0.8613]], grad_fn=<AddmmBackward0>)
prob: [0.04067358747124672, 0.7992402911186218, -0.8613470792770386]
labels: ['Educação', 'Política', 'Business']
scores: {'Educação': 0.04067358747124672, 'Política': 0.7992402911186218, 'Business': -0.8613470792770386}


## Tradução de Idioma


In [9]:
model_name = 'Helsinki-NLP/opus-mt-fr-en'
model = pipeline("translation", model=model_name)
model("Je suis étudiant")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

[{'translation_text': "I'm a student."}]

In [10]:
#model_name = 'Helsinki-NLP/opus-mt-en-fr'
#model = pipeline("translation", model=model_name)
#model("It is easy to translate languages with transformers")[0]['translation_text']

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_text = "Je suis étudiant"

#Tokenização manual
inputs = tokenizer(input_text, return_tensors="pt").input_ids
#Inferência manual
outputs = model.generate(input_ids=inputs)

print("Predição:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

Predição: ["I'm a student."]


## Perguntas e Respostas



In [12]:
model_name = 'distilbert-base-cased-distilled-squad'
question, context = "Where does Bob work?", "Several tech professionals work for Microsoft in USA."

In [13]:
#O pipeline lida com a tokenização, inferência e conversão do texto
model = pipeline("question-answering", model = model_name)
result = model(
    question = question,
    context = context,
)
result['answer']

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

'Microsoft'

In [14]:
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

#Tokenização manual
inputs = tokenizer(question, context, return_tensors="pt")
#Inferência manual
outputs = model(**inputs)

#Encontra o inicio e fim do índice onde a pontuação é máxima
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits)

#Conversão de tokens para texto
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx+1]))

print("Predição:", answer)

Predição: Microsoft
